In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import itertools
import sys

In [41]:
#Question 1
train_data_name=input("Enter the name of train data file [(ex) harris.dat] : ") # data name
coding_fm=int(input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): ")) # data separator
separator_fm={coding_fm ==1 : " "}.get(True, ",")
res_pos=int(input("Enter the column position of the response variable : [from 1 to p] : "))
header=input("Does the data have column header? (y/n) : ")
if(header=="y") : trdata=pd.read_csv(train_data_name, sep=separator_fm) # loading data
else : trdata=pd.read_csv(train_data_name, sep=separator_fm, header=None) # loading data
out_name=input("Enter the output file name to export [(ex) result.txt] : ")

# Extracting X and y
data = pd.DataFrame(trdata)
X= data.drop([data.columns[res_pos-1]], axis=1 )
Y= data.iloc[:, res_pos-1 ].values

Enter the name of train data file [(ex) harris.dat] : titanic.csv
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'): 2
Enter the column position of the response variable : [from 1 to p] : 4
Does the data have column header? (y/n) : y
Enter the output file name to export [(ex) result.txt] : result.txt


In [53]:
  def gini(Y):
    Y_1 =0; Y_2=0
    Z = np.unique(Y)
    for i in range(0,len(Y)):
      if Y[i] == Z[1]:
        Y_1 +=1
      else:
        Y_2 +=1 
    p1 = Y_1 / len(Y)
    p2 = Y_2 / len(Y)
    gini = 1- p1**2 -p2**2
  
    return gini


In [54]:
  def decisiontree( X, Y, method = 'CART'):
    if method == 'CART':
        A=[]
        B = np.unique(Y)
        for i in range(0,len(X.iloc[0])):
          if type(X.iloc[0,i]) == str:
            options = []
            for L in range(1,len(np.unique(X.iloc[:,i]))):
              for subset in itertools.combinations(np.unique(X.iloc[:,i]),L):
                subset = list(subset)
                options.append(subset)
            A.append(len(options))            
          else:
            A.append(len(X.iloc[:,i].sort_values().unique()))
        c = np.max(A)
        K = pd.DataFrame(index = range(0,len(X.iloc[0])), columns = range(0,c))
        I = pd.DataFrame(index = range(0,len(X.iloc[0])), columns = range(0,c))
        number =pd.DataFrame(index = range(0,len(X.iloc[0])), columns = range(0,c))
        for i in range(0,len(X.iloc[0])):
          Z=[]
          if type(X.iloc[0,i]) == str:
            options = []
            for L in range(1,len(np.unique(X.iloc[:,i]))):
              for subset in itertools.combinations(np.unique(X.iloc[:,i]),L):
                subset = list(subset)
                options.append(subset)
            for j in range(0,len(options)):
              condition = X.iloc[:,i].isin(options[j]) 
              l = len(Y[condition==True][Y[condition==True]==np.unique(Y)[1]])
              g = len(Y[condition==True])/len(Y)*gini(Y[condition==True])  + len(Y[condition==False])/len(Y)*gini(Y[condition==False])
              s = gini(Y) - g
              K[j][i] = s
              I[j][i] = np.array([options[j], list(np.delete(np.unique(X.iloc[:,i]),np.where(np.unique(X.iloc[:,i])==options[j])))]).flatten().tolist()
              number[j][i] = [[l,len(Y[condition == True])-l ], [len(Y[Y==np.unique(Y)[1]])-l, len(Y[Y==np.unique(Y)[0]])-(len(Y[condition == True])-l) ]]

          else:
            for j in range(0,len(X.iloc[:,i].sort_values().unique())-1):
              z1 = X.iloc[:,i].sort_values().unique()[j]
              z2 = X.iloc[:,i].sort_values().unique()[j+1]
              Z.append((z1+z2)/2)

              for k in range(0,len(Z)):
                condition = X.iloc[:,i]<=Z[k]
                l = len(Y[condition==True][Y[condition==True]==np.unique(Y)[1]])
                g = len(Y[condition==True])/len(Y)*gini(Y[condition==True])  + len(Y[condition==False])/len(Y)*gini(Y[condition==False])
                s = gini(Y) - g
                K[k][i] = s    
                I[k][i] = Z[k]   
                number[k][i] = [[l,len(Y[condition == True]-l) ], [len(Y[Y==np.unique(Y)[1]])-l, len(Y[Y==np.unique(Y)[0]])-(len(Y[condition == True])-l) ]]

    else:
        A=[]
        B = np.unique(Y)
        for i in range(0,len(X.iloc[0])):
          if type(X.iloc[0,i]) == str:
            A.append(1)            
          else:
            A.append(len(X.iloc[:,i].sort_values().unique()))
        c = np.max(A)
        K = pd.DataFrame(index = range(0,len(X.iloc[0])), columns = range(0,c))
        I = pd.DataFrame(index = range(0,len(X.iloc[0])), columns = range(0,c))
        number =pd.DataFrame(index = range(0,len(X.iloc[0])), columns = range(0,c))
        e = len(Y[Y==np.unique(Y)[1]])/len(Y)
        entropyY = -e*np.log2(e) -(1-e)*np.log2(1-e)

        for i in range(0,len(X.iloc[0])):
          if type(X.iloc[0,i]) == str:
            p=[]
            n=[]
            num=[]
            for j in range(0,len(np.unique(X.iloc[:,i]))):
              condition = (X.iloc[:,i]==np.unique(X.iloc[:,i])[j])
              l = len(Y[condition==True][Y[condition==True]==np.unique(Y)[1]])
              p1 = l/len(Y[condition==True])
              p.append((-p1*np.log2(p1) -(1-p1)*np.log2(1-p1))*(len(Y[condition==True])/(len(Y))))
              n.append(-(len(Y[condition==True])/len(Y))*np.log2(len(Y[condition==True])/len(Y)))
              num.append([l,len(Y[condition == True])-l ])
            K[0][i] = (entropyY-np.sum(p))/np.sum(n)
            I[0][i] = np.unique(X.iloc[:,i])
            number[0][i] = num

          else:
            Z=[]
            for j in range(0,len(X.iloc[:,i].sort_values().unique())-1):
              z1 = X.iloc[:,i].sort_values().unique()[j]
              z2 = X.iloc[:,i].sort_values().unique()[j+1]
              Z.append((z1+z2)/2)

              for k in range(0,len(Z)):
                condition = X.iloc[:,i]<=Z[k]
                l = len(Y[condition==True][Y[condition==True]==np.unique(Y)[1]])
                p1 = len(Y[condition==True][Y[condition==True]==np.unique(Y)[1]])/len(Y[condition==True])
                g = len(Y[condition==True])/len(Y)*(-p1*np.log2(p1))  + len(Y[condition==False])/len(Y)*(-(1-p1)*np.log2(1-p1))
                n= -len(Y[condition==True])/len(Y)*np.log2(len(Y[condition==True])/len(Y)) -(1-len(Y[condition==True])/len(Y))*np.log2(1-len(Y[condition==True])/len(Y))
                K[k][i] = (entropyY - g)/n   
                I[k][i] = Z[k]   
                number[k][i] = [[l,len(Y[condition == True]-l) ], [len(Y[Y==np.unique(Y)[1]])-l, len(Y[Y==np.unique(Y)[0]])-(len(Y[condition == True])-l) ]]
    
    
    K = K.astype(float)
    idx = []
    col = K.idxmax(axis=1)
    for i in range(0,len(X.iloc[0])):
      idx.append(K[col[i]][i])
    idx_r = np.argmax(idx) # 첫 노드를 나누는 변수
    idx_c = col[idx_r] 

   
    print('tree structure (', method, ')') 
    if len(Y[Y==np.unique(Y)[1]]) >= len(Y)/2:
      print( 'node 1 : yes  (', len(Y[Y==np.unique(Y)[1]]),',', len(Y) - len(Y[Y==np.unique(Y)[1]]), ')')
    else:
      print( 'node 1 : no  (', len(Y[Y==np.unique(Y)[1]]),',', len(Y) - len(Y[Y==np.unique(Y)[1]]), ')')
    for i in range(0,len(I[idx_c][idx_r])):
      if number[idx_c][idx_r][i][0]>= number[idx_c][idx_r][i][1]:
        print(f'node {i+2} :' 'class = ', I[idx_c][idx_r][i], ' yes (',number[idx_c][idx_r][i][0], ',' , number[idx_c][idx_r][i][1], ')' )
      else:
        print(f'node {i+2} :' 'class = ', I[idx_c][idx_r][i], ' no (',number[idx_c][idx_r][i][0], ',' , number[idx_c][idx_r][i][1], ')' )

    Y2= []
    if number[idx_c][idx_r][0][0]>= number[idx_c][idx_r][0][1]:
      for i in range(0,len(Y)):
        if X.iloc[:,idx_r][i] == I[idx_c][idx_r][0]:
          Y2.append(np.unique(Y)[1])
        else:
          Y2.append(np.unique(Y)[0])
    else:
        if X.iloc[:,idx_r][i] == I[idx_c][idx_r][1]:
          Y2.append(np.unique(Y)[1])
        else:
          Y2.append(np.unique(Y)[0])

    confusion_tst = confusion_matrix(Y,Y2)
    accu_tst = 0
    for i in range(len(np.unique(Y))):
      accu_tst = accu_tst + confusion_tst[i][i]
    accuracy_tst = accu_tst / X.shape[0]

    print('\n\nconfusion matrix (train)')
    print('----------------------------------')
    print('           predicted class \n Actual 1 ' ,confusion_tst[0], '\n class  2 ', confusion_tst[1])
    for i in range(2,len(np.unique(Y)) ):
      print(f'        {i+1} ', confusion_tst[i])
    print('model summary')
    print('----------------------------------')
    print('Overall accuracy = ' ,accuracy_tst)




In [55]:
sys.stdout = open(out_name,'w')
decisiontree(X,Y, method = 'C4_5')

In [56]:
print('')

In [57]:
decisiontree(X,Y, method = 'CART')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local